In [14]:
import wget

# wget.download('https://github.com/google-coral/edgetpu/raw/master/test_data/mobilenet_v2_1.0_224_quant_edgetpu.tflite', 'mobilenet_v2_1.0_224_quant_edgetpu.tflite')
wget.download('https://github.com/google-coral/edgetpu/raw/master/test_data/ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite', 'ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite')

'ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite'

In [21]:
from pycoral.utils import edgetpu
from pycoral.adapters import common
from pycoral.adapters import classify
import cv2

# Загрузка модели и подключение к Edge TPU
# model_path = "mobilenet_v2_1.0_224_quant_edgetpu.tflite"  # ваша квантованная модель
model_path = "models/ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite"  # ваша квантованная модель
interpreter = edgetpu.make_interpreter(model_path)
interpreter.allocate_tensors()  # выделение памяти

# Получение входных/выходных тензоров
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [23]:
# Открытие видеофайла или камеры
cap = cv2.VideoCapture('video/Видео с борта2 для вставки.mp4')  # или `0` для веб-камеры
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (640, 480))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Подготовка кадра (изменение размера, нормализация)
    input_size = input_details[0]['shape'][1:3]  # (224, 224) для MobileNet
    frame_resized = cv2.resize(frame, input_size)
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)  # BGR → RGB

    # Загрузка данных в интерпретатор
    common.set_input(interpreter, frame_rgb)

    # Запуск inference на Edge TPU
    interpreter.invoke()

    # Получение результатов
    classes = classify.get_classes(interpreter, top_k=3)  # топ-3 предсказания
    print(f"Предсказания: {classes}")

    # Отрисовка результатов (пример)
    for cls in classes:
        label = f"{cls.id}: {cls.score:.2f}"
        cv2.putText(frame, label, (10, 30 * (cls.id + 1)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    output.write(frame)  # внутри цикла

    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

output.release()
cap.release()
cv2.destroyAllWindows()

Предсказания: [Class(id=74, score=1.0028045), Class(id=72, score=0.9919947), Class(id=11, score=0.98764145)]
Предсказания: [Class(id=22, score=1.0018436), Class(id=78, score=0.9976006), Class(id=74, score=0.9969226)]
Предсказания: [Class(id=15, score=1.0149691), Class(id=79, score=1.0119388), Class(id=47, score=1.0000966)]
Предсказания: [Class(id=79, score=1.0368627), Class(id=43, score=1.0012789), Class(id=23, score=0.9931607)]
Предсказания: [Class(id=50, score=1.0057472), Class(id=74, score=1.0006002), Class(id=35, score=0.99750566)]
Предсказания: [Class(id=51, score=1.0037565), Class(id=30, score=1.0002632), Class(id=18, score=0.99661773)]
Предсказания: [Class(id=47, score=1.0008307), Class(id=26, score=0.99642587), Class(id=58, score=0.9929676)]
Предсказания: [Class(id=66, score=0.994433), Class(id=59, score=0.979134), Class(id=78, score=0.9772562)]
Предсказания: [Class(id=43, score=0.99750566), Class(id=42, score=0.9903316), Class(id=30, score=0.98809457)]
Предсказания: [Class(id=

In [ ]:
from pycoral.utils import edgetpu
from pycoral.adapters import common, classify
import cv2
import numpy as np

# 1. Загрузка модели
model_path = "models/mobilenet_v2_1.0_224_quant_edgetpu.tflite"
interpreter = edgetpu.make_interpreter(model_path)
interpreter.allocate_tensors()
input_size = common.input_size(interpreter)  # Получаем размер входа модели (224, 224)

In [16]:

# 2. Открытие видеофайла
cap = cv2.VideoCapture("input.mp4")

# 3. Настройка VideoWriter для сохранения результата
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Кодек для сохранения (может потребовать подбора для вашей системы)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # или 'avc1' для H.264
output = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 4. Предобработка кадра
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, input_size)

    # 5. Запуск модели на Edge TPU
    common.set_input(interpreter, frame_resized)
    interpreter.invoke()
    classes = classify.get_classes(interpreter, top_k=3)

    # 6. Отрисовка результатов
    for i, cls in enumerate(classes):
        label = f"{cls.id}: {cls.score:.2f}"
        cv2.putText(frame, label, (10, 30 * (i + 1)),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # 7. Запись кадра в выходной файл
    output.write(frame)

    # 8. Отображение процесса (опционально)
    cv2.imshow('Processing', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 9. Освобождение ресурсов
cap.release()
output.release()
cv2.destroyAllWindows()
print("Обработка видео завершена! Результат сохранен в output.mp4")

Обработка видео завершена! Результат сохранен в output.mp4


In [17]:
from pycoral.utils import edgetpu
from pycoral.adapters import common, detect
import cv2
import time

# 1. Загрузка модели детекции (например, SSD MobileNet для Edge TPU)
model_path = "models/mobilenet_v2_1.0_224_quant_edgetpu.tflite"  # ваша квантованная модель
# model_path = "ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite"
interpreter = edgetpu.make_interpreter(model_path)
interpreter.allocate_tensors()
input_size = common.input_size(interpreter)

In [18]:
# 2. Загрузка меток классов
with open("coco_labels.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]

In [19]:
# 3. Открытие видеофайла
cap = cv2.VideoCapture("video/Видео с борта2 для вставки.mp4")
if not cap.isOpened():
    print("Ошибка открытия видеофайла")
    exit()

In [20]:
# 4. Настройка VideoWriter
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter('output_detections.mp4', fourcc, fps, (frame_width, frame_height))

# Цвета для разных классов
colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0), (0, 255, 255), (255, 0, 255)]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Преобразование кадра для модели
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, input_size)

    # Запуск детекции
    common.set_input(interpreter, frame_resized)
    interpreter.invoke()
    objs = detect.get_objects(interpreter, score_threshold=0.5)

    # Масштабирование координат обратно к исходному размеру кадра
    scale_x = frame_width / input_size[0]
    scale_y = frame_height / input_size[1]

    # Отрисовка bounding boxes и меток
    for obj in objs:
        bbox = obj.bbox
        x0 = int(bbox.xmin * scale_x)
        y0 = int(bbox.ymin * scale_y)
        x1 = int(bbox.xmax * scale_x)
        y1 = int(bbox.ymax * scale_y)

        class_id = obj.id
        label = f"{labels[class_id]}: {obj.score:.2f}"
        color = colors[class_id % len(colors)]

        # Рисуем прямоугольник
        cv2.rectangle(frame, (x0, y0), (x1, y1), color, 2)

        # Рисуем подложку для текста
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        cv2.rectangle(frame, (x0, y0 - 25), (x0 + w, y0), color, -1)

        # Добавляем текст
        cv2.putText(frame, label, (x0, y0 - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    # Запись кадра в выходной файл
    output.write(frame)

    # Отображение процесса (опционально)
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
output.release()
cv2.destroyAllWindows()
print("Обработка завершена. Результат сохранен в output_detections.mp4")

IndexError: list index out of range